In [ ]:
import pandas as pd
import numpy as np

# Step 1: Read Input Data
def read_data():
    courses_df = pd.read_csv('courses.csv')
    teachers_df = pd.read_csv('teachers.csv')
    students_df = pd.read_csv('studentCourse.csv')
    student_course_df = pd.read_csv('studentNames.csv')
    return courses_df, teachers_df, students_df, student_course_df

Chromosome Representation

In [3]:
# Step 2: Chromosome Representation
#                 no of courses , no of slots for each exam , no of available timeslots
def initialize_chromosome(num_exams, num_genes_per_exam, num_timeslots):
    return np.random.randint(0, num_timeslots, size=(num_exams, num_genes_per_exam))

a = initialize_chromosome(3,5,7)
a


array([[4, 0, 2, 1, 1],
       [2, 5, 4, 1, 2],
       [6, 5, 0, 6, 6]])

Fitness Evaluation

In [9]:
import numpy as np

def evaluate_fitness(chromosome):
    fitness = 0
    
    # Hard Constraints
    # Constraint 1: An exam will be scheduled for each course.
    if not all(np.sum(chromosome, axis=1)):
        fitness += 1
    
    # Constraint 2: A student is enrolled in at least 3 courses. 
    # A student cannot give more than 1 exam at a time.
    for time_slot in range(chromosome.shape[1]):
        exams_at_time = chromosome[:, time_slot]
        if np.sum(exams_at_time) > 1:
            fitness += 1
        student_counts = np.sum(chromosome, axis=0)
        if any(student_counts < 3):
            fitness += 1
    
    # Constraint 3: Exam will not be held on weekends.
    if any(chromosome[:, -2:].flatten()):
        fitness += 1
    
    # Constraint 4: Each exam must be held between 9 am and 5 pm.
    if any(chromosome[:, :9].flatten()) or any(chromosome[:, 17:].flatten()):
        fitness += 1
    
    return fitness

Chromosome 1 fitness: 7


Genetic Algorithm

In [10]:
def selection(population, fitness_scores):
    selected_indices = np.random.choice(len(population), size=2, p=fitness_scores/np.sum(fitness_scores))
    return population[selected_indices]

def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1))
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

def mutation(chromosome, mutation_rate):
    for gene_index in range(len(chromosome)):
        if np.random.random() < mutation_rate:
            chromosome[gene_index] = np.random.randint(0, num_timeslots)
    return chromosome

Schedule Generation

In [11]:
# Step 5: Schedule Generation
def generate_schedule(num_exams, num_genes_per_exam, num_timeslots, population_size, max_generations, mutation_rate):
    population = [initialize_chromosome(num_exams, num_genes_per_exam, num_timeslots) for _ in range(population_size)]
    
    for generation in range(max_generations):
        fitness_scores = [evaluate_fitness(chromosome) for chromosome in population]
        parents = [selection(population, fitness_scores) for _ in range(population_size // 2)]
        offspring = [crossover(parent1, parent2) for parent1, parent2 in parents]
        offspring = [gene for pair in offspring for gene in pair]  # Flatten list
        mutated_offspring = [mutation(chromosome, mutation_rate) for chromosome in offspring]
        population = mutated_offspring

    best_chromosome = max(population, key=evaluate_fitness)
    return best_chromosome


In [13]:
# Step 7: Driver Code

# Read input data
courses_df, teachers_df, students_df, student_course_df = read_data()



# Define parameters
num_exams = len(courses_df)
num_genes_per_exam = 3  # Assuming each exam has 3 attributes (course, teacher, classroom)
num_timeslots = 5  # Number of available time slots for exams
population_size = 50
max_generations = 100
mutation_rate = 0.1

# Generate schedule
best_schedule = generate_schedule(num_exams, num_genes_per_exam, num_timeslots, population_size, max_generations, mutation_rate)

# Print best schedule
print("\nBest Schedule:")
print(best_schedule)


TypeError: only integer scalar arrays can be converted to a scalar index